In [ ]:
!pip install -qU \
  langchain-core \
  langchain-google-genai \
  langchain-community \
  scikit-image


In [ ]:
import getpass
import os

if not os.environ.get("GOOGLE_API_KEY"):
  os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter API key for Google Gemini: ")

In [ ]:
from langchain_google_genai import GoogleGenerativeAI


llm = GoogleGenerativeAI(model="gemini-2.0-flash", temperature=0.0) 

## Conversation Buffer Memory

In [ ]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(return_messages=True)

In [ ]:
memory.save_context(
    {"input": "Hi, my names in James"},
    {"output": "Hey James, what's up? I'm an AI model called Zeta."}
)
memory.save_context(
    {"input": "I am researching different types of conversational memory."},
    {"output": "That's interesting, what are some examples?"}
)
memory.save_context(
    {"input": "I am researching ConversationBufferMemory and ConversationBufferWindowMemory"},
    {"output": "That's interesting, what's thed difference"}
)
memory.save_context(
    {"input": "Buffer memory just store the enitre converation, right?"},
    {"output": "Thats makes sense, what about ConversationBufferWindowMemory"}
)
memory.save_context(
    {"input": "Buffer window memory stores last k messages, dropping the rest."},
    {"output": "Very cool!"}
)


In [ ]:
memory.load_memory_variables({})

In [ ]:
from langchain.chains import ConversationChain

chain = ConversationChain(
    llm = llm,
    memory = memory,
    verbose = True
)


In [ ]:
chain.invoke({"input": "What is my name again?"})

## ConversationBufferMemory with RunnableWithMessageHistory

In [ ]:
from langchain.prompts import (
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
    ChatPromptTemplate
)

system_prompt = "You are a helpful assistant called Zeta."

prompt_template = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(system_prompt),
    MessagesPlaceholder(variable_name="history"),
    HumanMessagePromptTemplate.from_template("{query}")   
])


In [ ]:
pipeline = (
    { 
        "query": lambda x: x["query"],
        "history": lambda x: x["history"]
    }
    | prompt_template
    | llm
)

In [ ]:
from langchain_core.chat_history import InMemoryChatMessageHistory

chat_map = {}
def get_chat_history(session_id: str) -> InMemoryChatMessageHistory:
    if session_id not in chat_map:
        # if session ID doesn't exist, create a new chat history
        chat_map[session_id] = InMemoryChatMessageHistory()
    return chat_map[session_id]

In [ ]:
from langchain_core.runnables.history import RunnableWithMessageHistory

pipeline_with_history = RunnableWithMessageHistory(
    pipeline,
    get_session_history=get_chat_history,
    input_messages_key="query",
    history_messages_key="history"
)

In [ ]:
pipeline_with_history.invoke(
    {"query": "Hi, my name is James"},
    config={"session_id": "id_123"}
)

In [ ]:
pipeline_with_history.invoke(
    {"query": "What is my name again?"},
    config={"session_id": "id_123"}
)

## ConversationBufferWindowMemory

In [ ]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(k=4, return_messages=True)

In [ ]:
memory.save_context(
    {"input": "Hi, my names in James"},
    {"output": "Hey James, what's up? I'm an AI model called Zeta."}
)
memory.save_context(
    {"input": "I am researching different types of conversational memory."},
    {"output": "That's interesting, what are some examples?"}
)
memory.save_context(
    {"input": "I am researching ConversationBufferMemory and ConversationBufferWindowMemory"},
    {"output": "That's interesting, what's thed difference"}
)
memory.save_context(
    {"input": "Buffer memory just store the enitre converation, right?"},
    {"output": "Thats makes sense, what about ConversationBufferWindowMemory"}
)
memory.save_context(
    {"input": "Buffer window memory stores last k messages, dropping the rest."},
    {"output": "Very cool!"}
)

In [ ]:
memory.load_memory_variables({})

In [ ]:
chain = ConversationChain(
    llm = llm,
    memory = memory,
    verbose = True
)

In [ ]:
chain.invoke({"input": "What is my name again"})

In [ ]:
from pydantic import BaseModel, Field
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.messages import BaseMessage

class BufferWindowMessageHistory(BaseChatMessageHistory, BaseModel):
    messages: list[BaseMessage] = Field(default_factory=list)
    k: int = Field(default_factory=int)
    
    
    def __init__(self, k:int):
        super().__init__(k=k)
        print(f"Initializing BufferWindowMessageHistory with k={k}")
        
    
    def add_messages(self, messages: list[BaseMessage]) -> None:
        """
        Add messages to the history, removing any messages beyond
        the last `k` messages
        """
        
        self.messages.extend(messages)
        self.messages = self.messages[-self.k:]
        
        
    def clear(self) -> None:
        """
        clear the history
        """
        self.messages = []       
        
        

In [ ]:
chat_map = {}
def get_chat_window_history(session_id: str, k: int = 4) -> BufferWindowMessageHistory:
    if session_id not in chat_map:
        chat_map[session_id] = BufferWindowMessageHistory(k=k)
    
    return chat_map[session_id]

In [ ]:
from langchain_core.runnables import ConfigurableFieldSpec

pipeline_cwbm = (
    { 
        "query": lambda x: x["query"],
        "history": lambda x: x["history"]
    }
    | prompt_template
    | llm
)
 

pipeline_with_history_cwbm = RunnableWithMessageHistory(
    pipeline_cwbm,
    get_session_history = get_chat_window_history,
    input_messages_key = "query",
    history_messages_key = "history",
    history_factory_config = [
        ConfigurableFieldSpec(
            id = "session_id",
            annotation = str,
            name = "Session ID",
            description = "The session ID to use for the chat history",
            default = "id_default"
        ),
        ConfigurableFieldSpec(
            id = "k",
            annotation = int,
            name = "k",
            description = "The number of messages to keep in the history",
            default = 4
        )
    ]
    
)

In [ ]:
pipeline_with_history_cwbm.invoke(
    {
        "query": "my name is James"  
    },
    config = {
        "configurable": {
            "session_id" : "id_k3",
            "k" : 3
        }
    }

)

In [ ]:
pipeline_with_history_cwbm.invoke(
    {
        "query": "I am learning langchain"  
    },
    config = {
        "configurable": {
            "session_id" : "id_k3",
            "k" : 3
        }
    }

)

In [ ]:
pipeline_with_history_cwbm.invoke(
    {
        "query": "My plan is to learn Langgraph and Google A2A, which one should I go"  
    },
    config = {
        "configurable": {
            "session_id" : "id_k3",
            "k" : 3
        }
    }

)

In [ ]:
chat_map["id_k3"].messages[2]

## Converasation Summary Memory

In [ ]:
from langchain.memory import ConversationSummaryMemory

summary_memory = ConversationSummaryMemory(llm = llm)

In [ ]:
summary_chain = ConversationChain(
    llm = llm,
    memory = summary_memory,
    verbose = True
)

In [ ]:
summary_chain.invoke({"input": "hello there my name in Muhammad"})
summary_chain.invoke({"input": "I am learning chain and exploring how it can be use"})
summary_chain.invoke({"input": "I am looking into ConversationBufferMemory and ConversationWindowBufferMemory"})
summary_chain.invoke({"input": "BufferMemory stores the entire conversation"})
summary_chain.invoke({"input": "BufferWindowMemory stores last k messages and dropped previous messages"})



In [ ]:
summary_chain.invoke({"input": "What is my name"})

## Conversation Summary Memory with Runnable Message History

In [ ]:
from langchain_core.messages import SystemMessage

In [ ]:
class ConversationSummaryMessageHistory(BaseChatMessageHistory, BaseModel): 
    messages: list[BaseMessage] = Field(default_factory=list)
    llm: GoogleGenerativeAI = Field(default_factory=GoogleGenerativeAI)
    
    def __init__(self, llm:GoogleGenerativeAI):
        super().__init__(llm=llm)
    
    def add_messages(self, messages: list[BaseMessage]) -> None:
        """Add messages to the history
        """
        self.messages.extend(messages)
        # construct the summary chat messages
        summary_prompt = ChatPromptTemplate.from_messages([
            SystemMessagePromptTemplate.from_template(
                "Given the existing conversation summary and the new messages, "
                "generate a new summary of the conversation. Ensuring to maintain "
                "as much relevant information as possible."
            ),
            HumanMessagePromptTemplate.from_template(
                "Existing conversation summary:\n{existing_summary}\n\n"
                "New messages:\n{messages}"
            )
        ])
        # format the messages and invoke the LLM
        new_summary = self.llm.invoke(
            summary_prompt.format_messages(
                existing_summary=self.messages,
                messages=[x.content for x in messages]
            )
        )
        # replace the existing history with a single system summary message
        self.messages = [SystemMessage(content=new_summary)]

    def clear(self) -> None:
        """Clear the history."""
        self.messages = []
    

In [ ]:
summary_chat_map = {}

def get_summary_chat_history(session_id: str, llm: GoogleGenerativeAI) -> ConversationSummaryMessageHistory:
    if session_id not in summary_chat_map:
        summary_chat_map[session_id] = ConversationSummaryMessageHistory(llm = llm)
    
    return summary_chat_map[session_id]

In [ ]:
pipeline_csmh = (
        { 
        "query": lambda x: x["query"],
        "history": lambda x: x["history"]
        }
    | prompt_template
    | llm
)

pipeline_with_summary_history = RunnableWithMessageHistory(
    pipeline_csmh,
    get_session_history = get_summary_chat_history,
    input_messages_key = "query",
    history_messages_key = "history",
    history_factory_config = [
        ConfigurableFieldSpec(
            id="session_id",
            annotation=str,
            name="Session ID",
            description="The session ID to use for the chat history",
            default="id_default",
        ),
        ConfigurableFieldSpec(
            id="llm",
            annotation=GoogleGenerativeAI,
            name="LLM",
            description="The LLM to use for the conversation summary",
            default=llm,
        )
        
    ]
)

In [ ]:
session_id = "csbm_123"

pipeline_with_summary_history.invoke(
    {
        "query": "Hi, my name is Muhammad"
    },
    config={
        "session_id": session_id, 
        "llm": llm
    }
)

In [ ]:
summary_chat_map[session_id].messages

In [ ]:
pipeline_with_summary_history.invoke(
    {
        "query": "I'm researching different type of conversational memory"
    },
    config={
        "session_id": session_id, 
        "llm": llm
    }
)

In [ ]:
for msg in [
    "I have been looking at ConversationBufferMemory and ConversationBufferWindowMemory.",
    "Buffer memory just stores the entire conversation",
    "Buffer window memory stores the last k messages, dropping the rest."
]:
    pipeline_with_summary_history.invoke(
        {"query": msg},
        config={"session_id": session_id, "llm": llm}
    )

In [ ]:
summary_chat_map[session_id].messages

In [ ]:
    pipeline_with_summary_history.invoke(
        {"query": "what is my name again"},
        config={"session_id": session_id, "llm": llm}
    )